In [ ]:
# ==============================================
# 1 Setup
# ==============================================
import os, random, zipfile, urllib.request
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.rcParams['figure.figsize'] = (10, 5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

torch.manual_seed(42)
np.random_seed(42)
random.seed(42)

# ==============================================
# 2 Load Caltech-256 Dataset (Resized)
# ==============================================
data_root = "./Caltech256"

# Ensure 3 channels
def ensure_3_channel(img):
    if isinstance(img, torch.Tensor):
        if img.shape[0] == 1:
            return img.repeat(3, 1, 1)
        return img
    else:
        if img.mode != 'RGB':
            return img.convert('RGB')
        return img

transform = T.Compose([
    T.Resize((64, 64)),
    T.Lambda(ensure_3_channel),
    T.ToTensor(),
])

try:
    dataset_full = torchvision.datasets.Caltech256(
        root=data_root,
        download=True,
        transform=transform
    )
    print(" Total Caltech-256 samples:", len(dataset_full))
except:
    class DummyCaltech256(torch.utils.data.Dataset):
        def __init__(self, size=2000):
            self.size = size
        def __len__(self):
            return self.size
        def __getitem__(self, idx):
            img = torch.rand(3, 64, 64)
            label = random.randint(0, 9)
            return img, label
    dataset_full = DummyCaltech256()
    print(" Using dummy dataset with 2000 samples")

# ==============================================
# 3 Strong NUI Mask Functions
# ==============================================
def generate_strong_nui_mask(h, w, strength=1.0, exponent=2.0):
    yy, xx = np.meshgrid(np.linspace(-1, 1, h), np.linspace(-1, 1, w), indexing='ij')
    angle = np.random.uniform(0, 2*np.pi)
    direction = np.cos(angle)*xx + np.sin(angle)*yy
    cx, cy = np.random.uniform(-0.5, 0.5, 2)
    r = np.sqrt((xx - cx)**2 + (yy - cy)**2)
    radial = 1 - np.clip(r, 0, 1)**exponent
    mask = 0.6*direction + 0.4*radial
    mask = (mask - mask.min()) / (mask.max() - mask.min())
    mask = 1 + strength * (mask - 0.5)
    mask = np.clip(mask, 0.1, 1.9).astype(np.float32)
    return mask

def apply_mask_to_tensor(img_tensor, mask):
    mask_tensor = torch.tensor(mask).unsqueeze(0)
    if mask_tensor.dim() == 3:
        mask_tensor = mask_tensor.unsqueeze(0)
    mask_tensor = F.interpolate(
        mask_tensor,
        size=img_tensor.shape[1:],
        mode='bilinear',
        align_corners=False
    ).squeeze(0)
    return img_tensor * mask_tensor

def apply_nui_to_img(img_tensor):
    h, w = img_tensor.shape[1], img_tensor.shape[2]
    mask = generate_strong_nui_mask(h, w)
    return apply_mask_to_tensor(img_tensor, mask)

# ==============================================
# 4 Subset for Speed (10 Classes)
# ==============================================
num_classes_to_use = 10
all_labels = []

for i in range(min(1000, len(dataset_full))):
    try:
        _, label = dataset_full[i]
        all_labels.append(label)
    except:
        continue

unique_labels = list(set(all_labels))
class_indices = random.sample(unique_labels, min(num_classes_to_use, len(unique_labels)))

class SubsetCaltech256(torch.utils.data.Dataset):
    def __init__(self, dataset, class_indices):
        self.dataset = dataset
        self.class_map = {c:i for i, c in enumerate(class_indices)}
        self.indices = [i for i, (_, lbl) in enumerate(dataset) if lbl in class_indices]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        img, lbl = self.dataset[real_idx]
        return img, self.class_map[lbl]

dataset = SubsetCaltech256(dataset_full, class_indices)
subset_size = min(2000, len(dataset))
subset_indices = list(range(subset_size))
random.shuffle(subset_indices)
dataset = Subset(dataset, subset_indices)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader  = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
num_classes = len(class_indices)
print(f"Train: {len(trainset)} | Test: {len(testset)} | Classes: {num_classes}")

# ==============================================
# 5 Tiny CNN Model (SqueezeNet)
# ==============================================
import torchvision.models as models

class SqueezeNetCustom(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.model = models.squeezenet1_1(pretrained=False)
        self.model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
        self.model.num_classes = num_classes

    def forward(self, x):
        return self.model(x)

# ==============================================
# 6 Evaluation Function
# ==============================================
def testing(model, loader, apply_nui=False):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            if apply_nui:
                imgs = torch.stack([apply_nui_to_img(img) for img in imgs])
            outputs = model(imgs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total

# ==============================================
# 7 Baseline Training (Clean)
# ==============================================
model_clean = SqueezeNetCustom(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_clean.parameters(), lr=0.001)
epochs = 3

for epoch in range(epochs):
    model_clean.train()
    running_loss = 0
    for imgs, labels in tqdm(trainloader, desc=f"Baseline Epoch {epoch+1}/{epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_clean(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss/len(trainloader):.4f}")
print("Baseline training done.\n")

# ==============================================
# 8 NUI-Augmented Training (Robust)
# ==============================================
def train_model_mixed(model, trainloader, optimizer, criterion, epochs, nui_ratio=0.8):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for imgs, labels in trainloader:
            imgs, labels = imgs.to(device), labels.to(device)
            mask = torch.rand(len(imgs)) < nui_ratio
            imgs_aug = [apply_nui_to_img(img) if mask[i] else img for i, img in enumerate(imgs)]
            imgs_aug = torch.stack(imgs_aug)
            optimizer.zero_grad()
            outputs = model(imgs_aug)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} Loss: {running_loss/len(trainloader):.4f}")

model_nui = SqueezeNetCustom(num_classes=num_classes).to(device)
optimizer = optim.Adam(model_nui.parameters(), lr=0.001)
train_model_mixed(model_nui, trainloader, optimizer, criterion, epochs, nui_ratio=0.8)
print("NUI-Augmented robust training complete.\n")

# ==============================================
# 9 Evaluate Models
# ==============================================
acc_clean     = testing(model_clean, testloader, apply_nui=False)
acc_nui       = testing(model_clean, testloader, apply_nui=True)
acc_clean_aug = testing(model_nui, testloader, apply_nui=False)
acc_nui_aug   = testing(model_nui, testloader, apply_nui=True)

acc_clean_pct     = acc_clean * 100
acc_nui_pct       = acc_nui * 100
acc_clean_aug_pct = acc_clean_aug * 100
acc_nui_aug_pct   = acc_nui_aug * 100

acc_clean_drop = acc_clean_pct - acc_nui_pct
print(f"Baseline Model:")
print(f"  Clean Test Accuracy: {acc_clean_pct:.2f}%")
print(f"  NUI Test Accuracy:   {acc_nui_pct:.2f}%")
print(f"  Accuracy Drop:       {acc_clean_drop:.2f}%\n")

acc_nui_increase = acc_nui_aug_pct - acc_nui_pct
robust_drop = acc_clean_aug_pct - acc_nui_aug_pct

print(f"Robust Model (NUI-Augmented Training):")
print(f"  Clean Test Accuracy: {acc_clean_aug_pct:.2f}%")
print(f"  NUI Test Accuracy:   {acc_nui_aug_pct:.2f}%")
print(f"  Accuracy Increase on NUI: {acc_nui_increase:.2f}%")

# ==============================================
# 10 Visualize NUI Effect
# ==============================================
# def visualize_nui_effect(num_images=4):
#     imgs, _ = next(iter(testloader))
#     fig, axes = plt.subplots(num_images, 3, figsize=(9, 3*num_images))
#     for i in range(num_images):
#         img = imgs[i]
#         mask = generate_strong_nui_mask(64, 64, strength=3.0, exponent=2.0)
#         img_nui = apply_mask_to_tensor(img, mask)
#         axes[i,0].imshow(img.permute(1,2,0)); axes[i,0].set_title("Original"); axes[i,0].axis("off")
#         axes[i,1].imshow(mask, cmap="gray"); axes[i,1].set_title("Mask"); axes[i,1].axis("off")
#         axes[i,2].imshow(img_nui.permute(1,2,0)); axes[i,2].set_title("NUI Applied"); axes[i,2].axis("off")
#     plt.tight_layout()
#     plt.show()
#
# visualize_nui_effect()


Using device: cpu
 Total Caltech-256 samples: 30607
Train: 839 | Test: 210 | Classes: 8


a:\college\College_project\AI_project\Non-Uniform_Illumination\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
a:\college\College_project\AI_project\Non-Uniform_Illumination\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Baseline Epoch 1/3:   0%|          | 0/14 [00:00<?, ?it/s]